In [1]:
!pip install -q transformers[sentencepiece] portalocker datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/KAGGLE DATASETS/intent /dataset.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['flags', 'utterance', 'category', 'intent'],
        num_rows: 21534
    })
})

In [5]:
intents = list(set(dataset["train"]["intent"]))
intents

['check_cancellation_fee',
 'review',
 'create_account',
 'complaint',
 'delivery_options',
 'payment_issue',
 'recover_password',
 'delivery_period',
 'place_order',
 'check_payment_methods',
 'set_up_shipping_address',
 'edit_account',
 'track_order',
 'change_order',
 'track_refund',
 'cancel_order',
 'contact_customer_service',
 'switch_account',
 'change_shipping_address',
 'newsletter_subscription',
 'contact_human_agent',
 'get_refund',
 'check_invoices',
 'registration_problems',
 'check_refund_policy',
 'get_invoice',
 'delete_account']

In [6]:
intent2id = {}
for i, intent in enumerate(intents):
  intent2id[intent] = i

intent2id

{'check_cancellation_fee': 0,
 'review': 1,
 'create_account': 2,
 'complaint': 3,
 'delivery_options': 4,
 'payment_issue': 5,
 'recover_password': 6,
 'delivery_period': 7,
 'place_order': 8,
 'check_payment_methods': 9,
 'set_up_shipping_address': 10,
 'edit_account': 11,
 'track_order': 12,
 'change_order': 13,
 'track_refund': 14,
 'cancel_order': 15,
 'contact_customer_service': 16,
 'switch_account': 17,
 'change_shipping_address': 18,
 'newsletter_subscription': 19,
 'contact_human_agent': 20,
 'get_refund': 21,
 'check_invoices': 22,
 'registration_problems': 23,
 'check_refund_policy': 24,
 'get_invoice': 25,
 'delete_account': 26}

In [7]:
from transformers import DebertaTokenizerFast, DebertaForSequenceClassification

In [8]:
model_ckpt = "microsoft/deberta-base"

In [9]:
tokenizer = DebertaTokenizerFast.from_pretrained(model_ckpt)

In [10]:
example_text = "Hello, I am Ronit!"

In [11]:
tokenizer.tokenize(example_text)

['Hello', ',', 'ĠI', 'Ġam', 'ĠRon', 'it', '!']

In [12]:
tokenizer(example_text)

{'input_ids': [1, 31414, 6, 38, 524, 5529, 405, 328, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
tokenizer.decode(tokenizer(example_text)["input_ids"])

'[CLS]Hello, I am Ronit![SEP]'

In [14]:
def vectorize_example(example):
  return tokenizer(example["utterance"], truncation=True, padding="max_length")

In [15]:
tokenized_dataset = dataset.map(vectorize_example, batched=True)

Map:   0%|          | 0/21534 [00:00<?, ? examples/s]

In [16]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['flags', 'utterance', 'category', 'intent', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 21534
    })
})

In [17]:
def preprocess_intents(example):
  return {"intent": intent2id[example["intent"]]}

In [18]:
tokenized_dataset = tokenized_dataset.map(preprocess_intents)

Map:   0%|          | 0/21534 [00:00<?, ? examples/s]

In [19]:
tokenized_dataset["train"]["intent"][0]

2

In [20]:
tokenized_dataset = tokenized_dataset.remove_columns(["utterance", "flags", "category"])
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['intent', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 21534
    })
})

In [21]:
tokenized_dataset = tokenized_dataset.rename_column("intent", "labels")
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 21534
    })
})

In [22]:
tokenized_dataset = tokenized_dataset["train"].train_test_split(test_size=0.3)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 15073
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6461
    })
})

In [23]:
tokenized_dataset.set_format("torch")

In [24]:
model = DebertaForSequenceClassification.from_pretrained(model_ckpt, num_labels=len(intents))
model

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'pooler.dense.bias', 'classifier.weight', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (

In [25]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
model = model.to(device)

In [27]:
from torch import optim
from torch import nn

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.AdamW(params=model.parameters(), lr=5e-5)

In [28]:
from torch.utils.data import DataLoader

train_loader = DataLoader(tokenized_dataset["train"], batch_size=16, shuffle=True)
test_loader = DataLoader(tokenized_dataset["test"], batch_size=16)

In [29]:
len(train_loader), 15073/16

(943, 942.0625)

In [30]:
for batch in train_loader:
  print(batch)
  break

{'labels': tensor([ 2,  6, 12,  4, 21,  3,  9, 12,  5,  0, 26,  4, 20, 19,  5,  5]), 'input_ids': tensor([[    1,  7424,    38,  ...,     0,     0,     0],
        [    1,  9178,   115,  ...,     0,     0,     0],
        [    1, 16026,  1437,  ...,     0,     0,     0],
        ...,
        [    1,   100, 23126,  ...,     0,     0,     0],
        [    1,   100,   300,  ...,     0,     0,     0],
        [    1,  4970,    41,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [31]:
for batch in train_loader:
  for k, v in batch.items():
    print(k, v.shape)
  break


labels torch.Size([16])
input_ids torch.Size([16, 512])
token_type_ids torch.Size([16, 512])
attention_mask torch.Size([16, 512])


In [32]:
EPOCHS = 5
def train(model, train_loader, optimizer, loss_fn):
  for epoch in range(EPOCHS):
    model.train()
    for i, batch in enumerate(train_loader):
      batch = {k: v.to(device) for k, v in batch.items()}
      optimizer.zero_grad()
      logits = model(**batch).logits

      loss = loss_fn(logits, batch["labels"])
      loss.backward()

      optimizer.step()
      if i%100==0:
        print(f"Loss {loss.item()}")


    print("-"*50)
    print(f"Epoch {epoch+1}: Loss {loss.item()}")
    print("-"*50)

In [33]:
train(model, train_loader, optimizer, loss_fn)

Loss 3.267984390258789
Loss 0.4958309531211853
Loss 0.04076509177684784
Loss 0.04781648889183998
Loss 0.3185155391693115
Loss 0.027044817805290222
Loss 0.050645798444747925
Loss 0.0049057318829
Loss 0.006608595605939627
Loss 0.0035680350847542286
--------------------------------------------------
Epoch 1: Loss 0.005667687859386206
--------------------------------------------------
Loss 0.001364329713396728
Loss 0.016623729839920998
Loss 0.002118633594363928
Loss 0.0022569745779037476
Loss 0.004014254081994295
Loss 0.007384610828012228
Loss 0.0019325865432620049
Loss 0.011638031341135502
Loss 0.06265978515148163
Loss 0.01052127406001091
--------------------------------------------------
Epoch 2: Loss 0.00051771110156551
--------------------------------------------------
Loss 0.001910041319206357
Loss 0.0011094120563939214
Loss 0.001143831294029951
Loss 0.0007769959047436714
Loss 0.002747751073911786
Loss 0.001303513185121119
Loss 0.19158165156841278
Loss 0.0560956709086895
Loss 0.004885

In [60]:
def compute_accuracy(y_preds, y):
  return (y_preds==y).mean()

In [43]:
def evaluate(model, loader, loss_fn):
  import numpy as np
  losses = []
  y, y_preds = [], []
  with torch.inference_mode():
    model.eval()

    for batch in loader:
      batch = {k: v.to(device) for k, v in batch.items()}

      logits = model(**batch).logits
      y_preds.append(logits.argmax(dim=-1).cpu().numpy())
      y.append(batch["labels"].cpu().numpy())


      loss = loss_fn(logits, batch["labels"])

      losses.append(loss.item())

  return np.mean(losses), (y_preds, y)

In [44]:
loss, preds_and_labels = evaluate(model, test_loader, loss_fn)

In [49]:
import numpy as np

y_preds, y = preds_and_labels
y_preds = np.concatenate(y_preds)
y = np.concatenate(y)
y_preds[0:10], y[0:10]

(array([25, 12, 19, 26,  5, 20,  5, 14, 22,  5]),
 array([25, 12, 19, 26,  5, 20,  5, 14, 22,  5]))

In [61]:
print(f"The accuracy of the test set is: {compute_accuracy(y_preds, y)}")

The accuracy of the test set is: 0.9959758551307847


In [64]:
id2intent = {id: intent for intent, id in intent2id.items()}

In [69]:
def predict(text, id2intent=id2intent):
  tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
  tokens = tokens.to(device)

  logits = model(**tokens).logits

  pred_label = logits.argmax(dim=-1).item()

  return id2intent[pred_label]

In [73]:
predict("Hello, I would like to create an account for your website. How should I proceed?")

'create_account'

In [74]:
predict("Good Afternoon, I have lost my password, how can i retrieve it?")

'recover_password'

In [75]:
predict("I donot remember my password so I cannot log in to the account. However I would like to delete my account permanently.")

'delete_account'

In [83]:
predict("I tried paying for the order online, but, I couldn't complete it. Kindly look into it.")

'payment_issue'

In [71]:
id2intent

{0: 'check_cancellation_fee',
 1: 'review',
 2: 'create_account',
 3: 'complaint',
 4: 'delivery_options',
 5: 'payment_issue',
 6: 'recover_password',
 7: 'delivery_period',
 8: 'place_order',
 9: 'check_payment_methods',
 10: 'set_up_shipping_address',
 11: 'edit_account',
 12: 'track_order',
 13: 'change_order',
 14: 'track_refund',
 15: 'cancel_order',
 16: 'contact_customer_service',
 17: 'switch_account',
 18: 'change_shipping_address',
 19: 'newsletter_subscription',
 20: 'contact_human_agent',
 21: 'get_refund',
 22: 'check_invoices',
 23: 'registration_problems',
 24: 'check_refund_policy',
 25: 'get_invoice',
 26: 'delete_account'}